In [ ]:
# DenseNet121 + 데이터 증강 (Resize / RandomHorizontalFlip / RandomRotation / RandomErasing)
# + Bounding Box --> Binary Mask

import os, re, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from torchvision.models import densenet121
import torchvision.transforms as transforms
import pandas as pd
from glob import glob
import cv2
from PIL import Image

# -------------------- 디바이스 설정 --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------- 하이퍼파라미터 --------------------
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"

batch_size = 16
num_epochs = 100
learning_rate = 1e-4
lambda_mga = 0.5

# -------------------- Attention 저장용 Hook --------------------
attn_feature = {}
def hook_fn(module, input, output):
    attn_feature['map'] = output

# -------------------- Transform --------------------
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])
val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# -------------------- Bounding Box --> Binary Mask --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    masks = []
    for bbox in bbox_list:
        mask = np.zeros(image_size, dtype=np.float32)
        x_min, y_min, x_max, y_max = bbox
        mask[y_min:y_max, x_min:x_max] = 1.0
        masks.append(mask)
    masks = np.stack(masks)
    masks = np.expand_dims(masks, axis=1)
    return torch.tensor(masks, dtype=torch.float32)

def load_bbox_dict(csv_path):
    df = pd.read_csv(csv_path)
    bbox_dict = {}
    for _, row in df.iterrows():
        pid = row['pid']
        slice_str = row['slice']
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])
        fname = f"{pid}_slice{slice_idx:03d}.npy"
        bbox = eval(row['bb'])
        bbox_dict.setdefault(fname, []).append(bbox)
    return bbox_dict

bbox_dict = load_bbox_dict(bbox_csv_path)

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except: return None

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        label = self.labels[idx]
        fname = os.path.basename(file_path)

        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = np.expand_dims(img, axis=-1)

        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)

        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(224, 224))
        else:
            mask = torch.zeros((1, 224, 224), dtype=torch.float32)

        return img, torch.tensor(label).long(), mask.squeeze(0)

    def __len__(self):
        return len(self.paths)

# -------------------- 학습 & 평가 루프 --------------------
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)

    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = densenet121(weights="IMAGENET1K_V1")
    model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.classifier = nn.Linear(model.classifier.in_features, 2)
    model.features.denseblock3.register_forward_hook(hook_fn)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "aug_resnet_cbam_mga.pth")

    metrics_log = []  # ⭐ 기록용 리스트

    for epoch in range(num_epochs):
        model.train(); epoch_loss = 0; correct = total = 0
        for imgs, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels, masks = imgs.to(device), labels.to(device), masks.to(device)
            outputs = model(imgs)
            ce_loss = criterion(outputs, labels)

            if 'map' in attn_feature:
                feat = attn_feature['map']
                attn_map = feat.mean(dim=1, keepdim=True)
                attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
                attn_loss = F.mse_loss(attn_map, masks)
                loss = ce_loss + lambda_mga * attn_loss
            else:
                loss = ce_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100. * correct / total
        train_loss = epoch_loss / len(train_loader)

        model.eval(); correct = total = 0; val_loss_accum = 0
        with torch.no_grad():
            for imgs, labels, masks in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss_accum += loss.item()
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = 100. * correct / total
        val_loss = val_loss_accum / len(val_loader)

        metrics_log.append({"epoch": epoch+1, "train_acc": train_acc, "train_loss": train_loss,
                            "val_acc": val_acc, "val_loss": val_loss})

        print(f"Train Acc: {train_acc:.2f}%, Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.2f}%, Val Loss: {val_loss:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # 로그 저장
    pd.DataFrame(metrics_log).to_csv("training_metrics_logged.csv", index=False)

    # 테스트
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path)); model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels, masks in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    run()



[Epoch 1]: 100%|██████████| 234/234 [01:41<00:00,  2.31it/s]


Train Acc: 67.04%, Train Loss: 0.6188, Val Acc: 69.50%, Val Loss: 0.5888
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:36<00:00,  6.46it/s]


Train Acc: 69.40%, Train Loss: 0.5836, Val Acc: 72.38%, Val Loss: 0.5675
✅ Saved best model!


[Epoch 3]: 100%|██████████| 234/234 [00:33<00:00,  7.07it/s]


...

Train Acc: 98.04%, Train Loss: 0.0528, Val Acc: 86.12%, Val Loss: 0.7227


[Epoch 99]: 100%|██████████| 234/234 [00:26<00:00,  8.82it/s]


Train Acc: 98.39%, Train Loss: 0.0438, Val Acc: 84.75%, Val Loss: 0.6151


[Epoch 100]: 100%|██████████| 234/234 [00:25<00:00,  9.08it/s]


Train Acc: 98.71%, Train Loss: 0.0316, Val Acc: 88.00%, Val Loss: 0.5763

📊 Test Evaluation:
✅ Test Accuracy: 88.88%
              precision    recall  f1-score   support

           0     0.8843    0.7782    0.8279       275
           1     0.8907    0.9467    0.9178       525

    accuracy                         0.8888       800
   macro avg     0.8875    0.8624    0.8728       800
weighted avg     0.8885    0.8888    0.8869       800

AUC: 0.9332
Confusion Matrix:
[[214  61]
 [ 28 497]]


In [ ]:
# 데이터 증강
# Resize / RandomHorizontalFlip / RandomRotation / RandomErasing

import os
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import gc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import cv2
from tqdm import tqdm
from torchvision.models import densenet121, DenseNet121_Weights

# --- 설정 ---
slice_root = "/data1/lidc-idri/slices"
batch_size = 16
num_epochs = 100
lr = 1e-4
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")

# --- 데이터 증강 Transform ---
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# --- 라벨 추출 ---
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3:
            return None
        return 1 if score >= 4 else 0
    except:
        return None

# --- 파일 로딩 및 3-way split ---
all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

# --- Dataset 정의 ---
class LIDCDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        img = np.load(file_path).astype(np.float32)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)

        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1)).float()  # [C, H, W]

        return img, torch.tensor(label).float()

# --- DataLoader ---
train_loader = DataLoader(LIDCDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(LIDCDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
test_loader = DataLoader(LIDCDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

# --- 모델 정의 ---
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.classifier = nn.Linear(model.classifier.in_features, 1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# --- 모델 저장 경로 설정 ---
base_dir = os.getcwd()
save_dir = os.path.join(base_dir, "pth")
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(save_dir, "best_model_densenet121.pth")
best_val_acc = 0.0

# --- 학습 루프 ---
for epoch in range(num_epochs):
    model.train()

    correct = 0
    train_loss = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = images.to(device)
        labels = labels.unsqueeze(1).to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs.data, 1)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f"[Train] Epoch: {epoch+1} Loss: {train_loss / len(train_loader):.4f} Accuracy: {(correct/total)*100:.2f}")
    torch.cuda.empty_cache()
    gc.collect()

    # --- Validation ---
    model.eval()
    correct = total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.sigmoid(outputs).squeeze()
            preds = (probs > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved!")

    torch.cuda.empty_cache()

# --- 테스트 ---
print("\n📊 Test Set Evaluation (Best Model 기준):")
model.load_state_dict(torch.load(save_path))
model.eval()
y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).long()
        y_probs.extend(probs.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

test_acc = (np.array(y_pred) == np.array(y_true)).mean() * 100
print(f"✅ Test Accuracy: {test_acc:.2f}%")
print(classification_report(y_true, y_pred, digits=4))

try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC 계산 실패: 클래스가 모두 있어야 합니다.")

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Densenet121 with 3-way split and evaluation metrics + memory cleanup

import os
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import gc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import cv2
from tqdm import tqdm
from torchvision.models import densenet121, DenseNet121_Weights

# --- 설정 ---
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {DEVICE}")

# --- 라벨 추출 ---
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3:
            return None
        return 1 if score >= 4 else 0
    except:
        return None

# --- 파일 로딩 및 3-way split ---
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

# --- Dataset 정의 ---
class LIDCDataset(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        img = np.load(file_path).astype(np.float32)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0)
        return torch.tensor(img).float(), torch.tensor(label).float()

# --- DataLoader ---
train_loader = DataLoader(LIDCDataset(train_files, train_labels), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(LIDCDataset(val_files, val_labels), batch_size=BATCH_SIZE)
test_loader = DataLoader(LIDCDataset(test_files, test_labels), batch_size=BATCH_SIZE)

# --- 모델 정의 ---
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.classifier = nn.Linear(model.classifier.in_features, 1)
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "best_model_densenet121.pth")
os.makedirs(os.path.dirname(save_path), exist_ok=True)
best_val_acc = 0.0

# --- 학습 루프 ---
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(DEVICE), labels.unsqueeze(1).to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {total_loss / len(train_loader):.4f}")
    torch.cuda.empty_cache()  # 메모리 정리

    gc.collect()
    torch.cuda.empty_cache()

    # --- Validation ---
    model.eval()
    correct = total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            probs = torch.sigmoid(outputs).squeeze()
            preds = (probs > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved!")

    torch.cuda.empty_cache()  # 검증 이후 메모리 정리

# --- 테스트 ---
print("\n📊 Test Set Evaluation (Best Model 기준):")
model.load_state_dict(torch.load(save_path))
model.eval()
y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images).squeeze()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).long()
        y_probs.extend(probs.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

test_acc = (np.array(y_pred) == np.array(y_true)).mean() * 100
print(f"✅ Test Accuracy: {test_acc:.2f}%")
print(classification_report(y_true, y_pred, digits=4))

try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC 계산 실패: 클래스가 모두 있어야 합니다.")

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

✅ Device: cuda


Epoch 1/100: 100%|██████████| 234/234 [00:24<00:00,  9.68it/s]


[Epoch 1] Loss: 0.5816
Validation Accuracy: 0.7200
✅ Best model saved!


Epoch 2/100: 100%|██████████| 234/234 [00:23<00:00, 10.14it/s]


[Epoch 2] Loss: 0.4496
Validation Accuracy: 0.7600
✅ Best model saved!


...

[Epoch 98] Loss: 0.0085
Validation Accuracy: 0.8512


Epoch 99/100: 100%|██████████| 234/234 [00:19<00:00, 11.77it/s]


[Epoch 99] Loss: 0.0037
Validation Accuracy: 0.8488


Epoch 100/100: 100%|██████████| 234/234 [00:19<00:00, 11.74it/s]


[Epoch 100] Loss: 0.0024
Validation Accuracy: 0.8475

📊 Test Set Evaluation (Best Model 기준):
✅ Test Accuracy: 86.25%
              precision    recall  f1-score   support

         0.0     0.8313    0.7527    0.7901       275
         1.0     0.8766    0.9200    0.8978       525

    accuracy                         0.8625       800
   macro avg     0.8540    0.8364    0.8439       800
weighted avg     0.8610    0.8625    0.8607       800

AUC: 0.9094
Confusion Matrix:
[[207  68]
 [ 42 483]]
